# 확률적 경사 하강법(SGD)을 이용한 행렬 분해

In [1]:
import numpy as np

In [26]:
# 원본 행렬 R생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정

R = np.array([[4 ,np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])

num_users, num_items = R.shape

K = 3

In [27]:
# P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력

np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))
P,Q

(array([[ 0.54144845, -0.2039188 , -0.17605725],
        [-0.35765621,  0.28846921, -0.76717957],
        [ 0.58160392, -0.25373563,  0.10634637],
        [-0.08312346,  0.48736931, -0.68671357]]),
 array([[-0.1074724 , -0.12801812,  0.37792315],
        [-0.36663042, -0.05747607, -0.29261947],
        [ 0.01407125,  0.19427174, -0.36687306],
        [ 0.38157457,  0.30053024,  0.16749811],
        [ 0.30028532, -0.22790929, -0.04096341]]))

In [28]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
  error = 0
  # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측행렬 R 생성
  full_pred_matrix = np.dot(P,Q.T)

  # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
  x_non_zero_idx = [non_zero[0] for non_zero in non_zeros]
  y_non_zero_idx = [non_zero[1] for non_zero in non_zeros]
  
  R_non_zeros = R[x_non_zero_idx, y_non_zero_idx]
  full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_idx, y_non_zero_idx]
  
  mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)

  rmse = np.sqrt(mse)
  
  return rmse
  

In [29]:
# R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
non_zeros = [(i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

# SGD 반복 업데이트 횟수
steps = 1000

# 학습률
learning_rate = 0.01

# R2 regulation 계수
r_lambda = 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
  for i, j , r in non_zeros:
    # 실제 값과 예측값 차이인 오류값을 구한다.
    eij = r - np.dot(P[i,:], Q[j,:].T)
    # Regularization을 반영한 SGD 업데이트 공식 적용
    P[i,:] = P[i,:] + learning_rate * (eij * Q[j,:] - r_lambda*P[i,:])
    Q[j,:] = Q[j,:] + learning_rate * (eij * P[i,:] - r_lambda*Q[j,:])
  rmse = get_rmse(R, P, Q, non_zeros)
  if (step % 50) == 0:
    print('### iteration step: ', step,'rmse: ',rmse)

### iteration step:  0 rmse:  3.2388050277987723
### iteration step:  50 rmse:  0.4876723101369647
### iteration step:  100 rmse:  0.15643403848192458
### iteration step:  150 rmse:  0.07455141311978064
### iteration step:  200 rmse:  0.043252267985793146
### iteration step:  250 rmse:  0.029248328780879226
### iteration step:  300 rmse:  0.022621116143829507
### iteration step:  350 rmse:  0.019493636196525232
### iteration step:  400 rmse:  0.018022719092132773
### iteration step:  450 rmse:  0.01731968595344283
### iteration step:  500 rmse:  0.016973657887570985
### iteration step:  550 rmse:  0.01679680459589558
### iteration step:  600 rmse:  0.016701322901884634
### iteration step:  650 rmse:  0.016644736912476574
### iteration step:  700 rmse:  0.016605910068210012
### iteration step:  750 rmse:  0.01657420047570488
### iteration step:  800 rmse:  0.01654431582921612
### iteration step:  850 rmse:  0.016513751774735037
### iteration step:  900 rmse:  0.01648146573819507
### ite

In [30]:
pred_matrix = np.dot(P, Q.T)
print('예측행렬:\n', np.round(pred_matrix, 3))

예측행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


### 예측행렬이 원본 행렬 R과 비교해 NaN이 아닌 값은 큰 차이가 나지 않으며,  
### NaN인 값은 새로운 예측값으로 채워진것을 확인할 수 있다.